In [3]:
# necessary functions
def parseTable(fn, sep, header = False,excel = False):
    '''takes in a table where columns are separated by a given symbol and outputs
    a nested list such that list[row][col]
    example call:
    table = parseTable('file.txt','\t')
    '''
    fh = open(fn)
    lines = fh.readlines()
    fh.close()
    if excel:
        lines = lines[0].split('\r')
    if lines[0].count('\r') > 0:
        lines = lines[0].split('\r')
    table = []
    if header == True:
        lines =lines[1:]
    for i in lines:
        table.append(i[:-1].split(sep))

    return table


def unParseTable(table, output, sep):
    '''takes in a table generated by parseTable and writes it to an output file
    takes as parameters (table, output, sep), where sep is how the file is delimited
    example call unParseTable(table, 'table.txt', '\t') for a tab del file
    '''
    fh_out = open(output,'w')
    if len(sep) == 0:
        for i in table:
            fh_out.write(str(i))
            fh_out.write('\n')
    else:
        for line in table:
            line = [str(x) for x in line]
            line = sep.join(line)

            fh_out.write(line)
            fh_out.write('\n')
            
    print(f'write table to {output}')

    fh_out.close()
    

'''uniquify function by Peter Bengtsson Used under a creative commons license
sourced from  here: http://www.peterbe.com/plog/uniqifiers-benchmark
'''
def uniquify(seq, idfun=None):
    if idfun is None:
        def idfun(x): return x
    seen = {}
    result = []
    for item in seq:
        marker = idfun(item)
        if marker in seen: continue
        seen[marker] = 1
        result.append(item)
    return result


def makeSearchLocus(locus,upSearch,downSearch):
    '''takes a locus and expands it by a fixed upstream/downstream amount. spits out the new larger locus
    '''
    if locus.sense() == '-':
        searchLocus = Locus(locus.chr(),locus.start()-downSearch,locus.end()+upSearch,locus.sense(),locus.ID())
    else:
        searchLocus = Locus(locus.chr(),locus.start()-upSearch,locus.end()+downSearch,locus.sense(),locus.ID())
    return searchLocus


def importRefseq(refseqFile, returnMultiples = False):

    '''
    opens up a refseq file downloaded by UCSC
    '''
    refseqTable = parseTable(refseqFile,'\t')
    refseqDict = {}
    ticker = 1
    for line in refseqTable[1:]:
        if refseqDict.__contains__(line[1]):
            refseqDict[line[1]].append(ticker)
        else:
            refseqDict[line[1]] = [ticker]
        ticker = ticker + 1

    multiples = []
    for i in refseqDict:
        if len(refseqDict[i]) > 1:
            multiples.append(i)

    if returnMultiples == True:
        return refseqTable,refseqDict,multiples
    else:
        return refseqTable,refseqDict

    
def getTSSs(geneList,refseqTable,refseqDict):
    if len(geneList) == 0:
        refseq = refseqTable
    else:
        refseq = refseqFromKey(geneList,refseqDict,refseqTable)
    TSS = []
    for line in refseq:
        if line[3] == '+':
            TSS.append(line[4])
        if line[3] == '-':
            TSS.append(line[5])
    TSS = map(int,TSS)
    return TSS


def refseqFromKey(refseqKeyList,refseqDict,refseqTable):
    typeRefseq = []
    for name in refseqKeyList:
        if refseqDict.__contains__(name):
            typeRefseq.append(refseqTable[refseqDict[name][0]])
    return typeRefseq


def makeStartDict(annotFile,geneList = []):
    '''makes a dictionary keyed by refseq ID that contains information about 
    chrom/start/stop/strand/common name
    '''

    if type(geneList) == str:
        geneList = parseTable(geneList,'\t')
        geneList = [line[0] for line in geneList]
            
    if annotFile.upper().count('REFSEQ') == 1:
        refseqTable,refseqDict = importRefseq(annotFile)
        if len(geneList) == 0:
            geneList = refseqDict.keys()
        startDict = {}
        for gene in geneList:
            if refseqDict.__contains__(gene) == False:
                continue
            startDict[gene]={}
            startDict[gene]['sense'] = refseqTable[refseqDict[gene][0]][3]
            startDict[gene]['chr'] = refseqTable[refseqDict[gene][0]][2]
            startDict[gene]['start'] = [*getTSSs([gene],refseqTable,refseqDict)]
            if startDict[gene]['sense'] == '+':
                startDict[gene]['end'] =[int(refseqTable[refseqDict[gene][0]][5])]
            else:
                startDict[gene]['end'] = [int(refseqTable[refseqDict[gene][0]][4])]
            startDict[gene]['name'] = refseqTable[refseqDict[gene][0]][12]
    return startDict


def makeTSSLocus(gene,startDict,upstream,downstream):
    '''given a startDict, make a locus for any gene's TSS w/ upstream and downstream windows
    '''
    
    start = startDict[gene]['start'][0]
    if startDict[gene]['sense'] =='-':
        return Locus(startDict[gene]['chr'],start-downstream,start+upstream,'-',gene)
    else:
        return Locus(startDict[gene]['chr'],start-upstream,start+downstream,'+',gene)
    
    
def fetchSeq(directory,chrom,start,end,UCSC=False,lineBreaks=True,header = True):
    '''function that fetches a sequence from a genome directory
    directory that contains individual chrom fasta files
    '''
    fn = directory + chrom + '.fa'
    fh = open(fn,'r')
    headerOffset = 0
    nStart = 0
    nEnd = 0
    if header:
        fh.seek(0)
        headerOffset = len(fh.readline())
    if lineBreaks:

        nStart = int((start-1)/50)
        nEnd = int((end-1)/50)
    if UCSC:
        fh.seek((start+nStart+headerOffset))
    else:
        fh.seek((start-1+nStart+headerOffset))
    span = ((end+nEnd-1)-(start+nStart-1))

    read = fh.read(span)
    if lineBreaks:
        read = read.replace('\n','')

    return read
    fh.close()

    
class Locus:
    __chrDict = dict()
    __senseDict = {'+':'+', '-':'-', '.':'.'}
    def __init__(self,chr,start,end,sense,ID='',score=0):
        coords = [int(start),int(end)]
        # coords = [start, end]
        coords.sort(reverse=False)
        if not(self.__chrDict.__contains__(chr)): self.__chrDict[chr] = chr
        self._chr = self.__chrDict[chr]
        self._sense = self.__senseDict[sense]
        self._start = int(coords[0])
        self._end = int(coords[1])
        self._ID = ID
        self._score = score
    def ID(self): return self._ID
    def chr(self): return self._chr
    def start(self): return self._start
    def end(self): return self._end
    def len(self): return self._end - self._start + 1
    def score(self): return self._score
    def getAntisenseLocus(self):
        if self._sense=='.': return self
        else:
            switch = {'+':'-', '-':'+'}
            return Locus(self._chr,self._start,self._end,switch[self._sense])
    def coords(self): return [self._start,self._end]
    def sense(self): return self._sense
    def overlaps(self,otherLocus):
        if self.chr()!=otherLocus.chr(): return False
        elif not(self._sense=='.' or \
                 otherLocus.sense()=='.' or \
                 self.sense()==otherLocus.sense()): return False
        elif self.start() > otherLocus.end() or otherLocus.start() > self.end(): return False
        else: return True
    def contains(self,otherLocus):
        if self.chr()!=otherLocus.chr(): return False
        elif not(self._sense=='.' or \
                 otherLocus.sense()=='.' or \
                 self.sense()==otherLocus.sense()): return False
        elif self.start() > otherLocus.start() or otherLocus.end() > self.end(): return False
        else: return True
    def overlapsAntisense(self,otherLocus):
        return self.getAntisenseLocus().overlaps(otherLocus)
    def containsAntisense(self,otherLocus):
        return self.getAntisenseLocus().contains(otherLocus)
    def __hash__(self): return self._start + self._end
    def __eq__(self,other):
        if self.__class__ != other.__class__: return False
        if self.chr()!=other.chr(): return False
        if self.start()!=other.start(): return False
        if self.end()!=other.end(): return False
        if self.sense()!=other.sense(): return False
        return True
    def __ne__(self,other): return not(self.__eq__(other))
    def __str__(self): return self.chr()+'('+self.sense()+'):'+'-'.join(map(str,self.coords()))
    def plotStr(self): return self.chr() + ':' + self.sense() + ':' + '-'.join(map(str,self.coords()))
    def checkRep(self):
        pass
    def gffLine(self): return [self.chr(),self.ID(),'',self.start(),self.end(),'',self.sense(),'',self.ID()]
    
    
class LocusCollection:
    def __init__(self,loci,windowSize):
        self.__chrToCoordToLoci = dict()
        self.__loci = dict()
        self.__winSize = windowSize
        for lcs in loci: self.__addLocus(lcs)

    def __addLocus(self,lcs):
        if not(self.__loci.__contains__(lcs)):
            self.__loci[lcs] = None
            if lcs.sense()=='.': chrKeyList = [lcs.chr()+'+', lcs.chr()+'-']
            else: chrKeyList = [lcs.chr()+lcs.sense()]
            for chrKey in chrKeyList:
                if not(self.__chrToCoordToLoci.__contains__(chrKey)): self.__chrToCoordToLoci[chrKey] = dict()
                for n in self.__getKeyRange(lcs):
                    if not(self.__chrToCoordToLoci[chrKey].__contains__(n)): self.__chrToCoordToLoci[chrKey][n] = []
                    self.__chrToCoordToLoci[chrKey][n].append(lcs)
    def __getKeyRange(self,locus):
        start = int(locus.start() / self.__winSize)
        end = int(locus.end() / self.__winSize) + 1
        return range(start,end)
    def __len__(self): return len(self.__loci)
        
    def append(self,new): self.__addLocus(new)
    def extend(self,newList):
        for lcs in newList: self.__addLocus(lcs)
    def hasLocus(self,locus):
        return self.__loci.__contains__(locus)
    def remove(self,old):
        if not(self.__loci.__contains__(old)): raise ValueError("requested locus isn't in collection")
        del self.__loci[old]
        if old.sense()=='.': senseList = ['+','-']
        else: senseList = [old.sense()]
        for k in self.__getKeyRange(old):
            for sense in senseList:
                self.__chrToCoordToLoci[old.chr()+sense][k].remove(old)
    def getWindowSize(self): return self.__winSize
    def getLoci(self): return self.__loci.keys()
    def getChrList(self):
        tempKeys = dict()
        for k in self.__chrToCoordToLoci.keys(): tempKeys[k[:-1]] = None
        return tempKeys.keys()
            
    def __subsetHelper(self,locus,sense):
        sense = sense.lower()
        if ['sense','antisense','both'].count(sense)!=1:
            raise ValueError("sense command invalid: '"+sense+"'.")
        matches = dict()
        senses = ['+','-']
        if locus.sense()=='.' or sense=='both': lamb = lambda s: True
        elif sense=='sense': lamb = lambda s: s==locus.sense()
        elif sense=='antisense': lamb = lambda s: s!=locus.sense()
        else: raise ValueError("sense value was inappropriate: '"+sense+"'.")
        for s in filter(lamb, senses):
            chrKey = locus.chr()+s
            if self.__chrToCoordToLoci.__contains__(chrKey):
                for n in self.__getKeyRange(locus):
                    if self.__chrToCoordToLoci[chrKey].__contains__(n):
                        for lcs in self.__chrToCoordToLoci[chrKey][n]:
                            matches[lcs] = None
        return matches.keys()
    def getOverlap(self,locus,sense='sense'):
        matches = self.__subsetHelper(locus,sense)
        realMatches = dict()
        if sense=='sense' or sense=='both':
            for i in filter(lambda lcs: lcs.overlaps(locus), matches):
                realMatches[i] = None
        if sense=='antisense' or sense=='both':
            for i in filter(lambda lcs: lcs.overlapsAntisense(locus), matches):
                realMatches[i] = None 
        return realMatches.keys()
    def getContained(self,locus,sense='sense'):
        matches = self.__subsetHelper(locus,sense)
        realMatches = dict()
        if sense=='sense' or sense=='both':
            for i in filter(lambda lcs: locus.contains(lcs), matches):
                realMatches[i] = None
        if sense=='antisense' or sense=='both':
            for i in filter(lambda lcs: locus.containsAntisense(lcs), matches):
                realMatches[i] = None
        return realMatches.keys()
    def getContainers(self,locus,sense='sense'):
        matches = self.__subsetHelper(locus,sense)
        realMatches = dict()
        if sense=='sense' or sense=='both':
            for i in filter(lambda lcs: lcs.contains(locus), matches):
                realMatches[i] = None
        if sense=='antisense' or sense=='both':
            for i in filter(lambda lcs: lcs.containsAntisense(locus), matches):
                realMatches[i] = None
        return realMatches.keys()
    def stitchCollection(self,stitchWindow=1,sense='both'):
        locusList = self.getLoci()
        oldCollection = LocusCollection(locusList,500)
        stitchedCollection = LocusCollection([],500)
        for locus in locusList:
            if oldCollection.hasLocus(locus):
                oldCollection.remove(locus)
                overlappingLoci = oldCollection.getOverlap(Locus(locus.chr(),locus.start()-stitchWindow,locus.end()+stitchWindow,locus.sense(),locus.ID()),sense)
                stitchTicker = 1
                while len(overlappingLoci) > 0:
                    stitchTicker+=len(overlappingLoci)
                    overlapCoords = locus.coords()
                    for overlappingLocus in overlappingLoci:
                        overlapCoords+=overlappingLocus.coords()
                        oldCollection.remove(overlappingLocus)
                    if sense == 'both':
                        locus = Locus(locus.chr(),min(overlapCoords),max(overlapCoords),'.',locus.ID())
                    else:
                        locus = Locus(locus.chr(),min(overlapCoords),max(overlapCoords),locus.sense(),locus.ID())
                    overlappingLoci = oldCollection.getOverlap(Locus(locus.chr(),locus.start()-stitchWindow,locus.end()+stitchWindow,locus.sense()),sense)
                locus._ID = '%s_%s_lociStitched' % (stitchTicker,locus.ID())
                stitchedCollection.append(locus)
            else:
                continue
        return stitchedCollection
    def getLoci(self): return self.__loci.keys()

In [4]:
from subprocess import call
import networkx as nx

In [5]:
# input files
## for refseqToNameDict
annotationFile = './CRCmapper/CRCmapper_package/annotation/hg19_refseq_NM.ucsc'
annotTable = parseTable(annotationFile, '\t')

## refseqToNameDict
refseqToNameDict = {}
for line in annotTable[1:]:
    gid = line[1]
    genename = line[12].upper()
    refseqToNameDict[gid] = genename
# print(dict(list(refseqToNameDict.items())[0:5]))

## SuperTable
superFile = './ROSE/2.rose_Beta_narrow.05_broad.1_incltss/Beta_peaks_AllEnhancers.table.txt'
superTable = parseTable(superFile, '\t')

## SuperTable changed to tadTable
tadFile = './hg19.GSE63525_GM12878_50K.TopDom_10.bed'
tadTable = parseTable(tadFile, '\t')
for i in range(1, len(tadTable)+1):
    tadTable[i-1].append(f'hg19_TAD_{i}')
### chr start end type name 
## 3139

## enhancerNumber
## enhancerNumber = 500

## expressionTable
# expressionFile = './CRCmapper/2-2.crc_Beta_narrow.05_broad.1_incltss_cutoff500/matrix.gff'
# expressionFile = './CRCmapper/1-1.crc_HI-32_K27ac_incltss/matrix.gff'
# expressionTable = parseTable(expressionFile, '\t')

## TFfile
TFfile = './CRCmapper/CRCmapper_package/TFlist_NMid_hg.txt'
TFtable = parseTable(TFfile, '\t')
TFlist = [line[0] for line in TFtable]
TFlistGene = [line[1] for line in TFtable]

## subpeaks
subpeaks = './ChIP-seq/Meissner/macs2_Beta_narrow.05_broad.1/Beta_peaks.broadPeak'
# subpeaks = './crup/islets.HI-32.CRUP.singleEnh.bed'

## genomeDirectory FASTA (DNA sequences)
genomeDirectory = './CRCmapper/hg19/'

## motifExtension
motifExtension = 500

## expCutoff: top 2/3 of the genes are considered to be expressed
## expCutoff = 33

## Transfac.v$ZIC2_01 ZIC2
motifConvertFile = './CRCmapper/CRCmapper_package/MotifDictionary.txt'
# motifDatabase = parseTable(motifConvertFile, '\t')

## PWM (motifDatabaseFile)
PWMfile = './CRCmapper/CRCmapper_package/VertebratePWMs.txt'

In [17]:

# 2-2-TAD-TRAP.Beta_narrow.05_broad.1_incltss_cutoff500_ext500
## enhancerNumber
Enumber = 500
## motifExtension
motifExtension = 500
## working directory
projectFolder = f'./CRCmapper/2-2-TAD-TRAP.crc_Beta_narrow.05_broad.1_incltss_cutoff{Enumber}_ext{motifExtension}/'
projectName = 'Beta'
## SuperTable
superFile = './ROSE/2.rose_Beta_narrow.05_broad.1_incltss/Beta_peaks_AllEnhancers.table.txt'
superTable = parseTable(superFile, '\t')
## subpeak
subpeaks = './ChIP-seq/Meissner/macs2_Beta_narrow.05_broad.1/Beta_peaks.broadPeak'
## expressionTable
expressedNMfile = f'./CRCmapper/2-2.crc_Beta_narrow.05_broad.1_incltss_cutoff{Enumber}_ext{motifExtension}/Beta_EXPRESSED_TRANSCRIPTS.txt'


'''
# 2-2-TAD-TRAP.Beta_narrow.05_broad.1_incltss_cutoff500_ext0
## enhancerNumber
Enumber = 500
## motifExtension
motifExtension = 0
## working directory
projectFolder = f'./CRCmapper/2-2-TAD-TRAP.crc_Beta_narrow.05_broad.1_incltss_cutoff{Enumber}_ext{motifExtension}/'
projectName = 'Beta'
## SuperTable
superFile = './ROSE/2.rose_Beta_narrow.05_broad.1_incltss/Beta_peaks_AllEnhancers.table.txt'
superTable = parseTable(superFile, '\t')
## subpeak
subpeaks = './ChIP-seq/Meissner/macs2_Beta_narrow.05_broad.1/Beta_peaks.broadPeak'
## expressionTable
expressedNMfile = f'./CRCmapper/2-2.crc_Beta_narrow.05_broad.1_incltss_cutoff{Enumber}_ext500/Beta_EXPRESSED_TRANSCRIPTS.txt'
'''

'''
# 3-1-TAD-TRAP.crc_CRUP_HI-32_K27ac_incltss_peakCutoff.8_cutoffSE_ext500
## enhancerNumber
Enumber = 'SE'  # 668
## motifExtension
motifExtension = 500
## working directory
projectFolder = f'./CRCmapper/3-1-TAD-TRAP.crc_CRUP_HI-32_K27ac_incltss_peakCutoff.8_cutoff{Enumber}_ext{motifExtension}/'
projectName = 'CRUP_HI-32_K27ac'
## SuperTable
superFile = './ROSE/3-1.rose_CRUP_HI-32_K27ac_incltss_peakCutoff.8/islets_AllEnhancers.table.txt'
superTable = parseTable(superFile, '\t')
## subpeak
subpeaks = './crup/islets.HI-32.CRUP.singleEnh_peakCutoff.8.bed'
## expressionTable
expressedNMfile = f'./CRCmapper/3-1.crc_CRUP_HI-32_K27ac_incltss_peakCutoff.8_cutoff{Enumber}_ext500/H3K27ac_HI-32_EXPRESSED_TRANSCRIPTS.txt'
'''

'''
# 3-1-TAD-TRAP.crc_CRUP_HI-32_K27ac_incltss_peakCutoff.8_cutoffSE_ext0
## enhancerNumber
Enumber = 'SE'  # 668
## motifExtension
motifExtension = 0
## working directory
projectFolder = f'./CRCmapper/3-1-TAD-TRAP.crc_CRUP_HI-32_K27ac_incltss_peakCutoff.8_cutoff{Enumber}_ext{motifExtension}/'
projectName = 'CRUP_HI-32_K27ac'
## SuperTable
superFile = './ROSE/3-1.rose_CRUP_HI-32_K27ac_incltss_peakCutoff.8/islets_AllEnhancers.table.txt'
superTable = parseTable(superFile, '\t')
## subpeak
subpeaks = './crup/islets.HI-32.CRUP.singleEnh_peakCutoff.8.bed'
## expressionTable
expressedNMfile = f'./CRCmapper/3-1.crc_CRUP_HI-32_K27ac_incltss_peakCutoff.8_cutoff{Enumber}_ext{motifExtension}/H3K27ac_HI-32_EXPRESSED_TRANSCRIPTS.txt'
'''

'''
# 2-1-TAD-TRAP.macs2_HI-32_K27ac_narrow.05_broad.05_incltss_cutoffSE_ext500
## enhancerNumber
Enumber = 'SE'
## motifExtension
motifExtension = 500
## working directory
projectFolder = f'./CRCmapper/2-1-TAD-TRAP.crc_macs2_HI-32_K27ac_narrow.05_broad.05_incltss_cutoff{Enumber}_ext{motifExtension}/'
projectName = 'macs2_HI-32_K27ac'
## SuperTable
superFile = './ROSE/2-1.rose_macs2_HI-32_K27ac_narrow.05_broad.05_incltss/H3K27ac_HI-32_peaks_AllEnhancers.table.txt'
superTable = parseTable(superFile, '\t')
## subpeak
subpeaks = './ChIP-seq/Meissner/macs2_HI-32_K27ac_narrow.05_broad.05/H3K27ac_HI-32_peaks.broadPeak'
## expressionTable
expressedNMfile = f'./CRCmapper/2-1.crc_macs2_HI-32_K27ac_narrow.05_broad.05_incltss_cutoff{Enumber}_ext{motifExtension}/H3K27ac_HI-32_EXPRESSED_TRANSCRIPTS.txt'
'''

'''
# 2-1. macs2_HI-32_K27ac_narrow.05_broad.05_incltss_cutoffSE_ext0
## enhancerNumber
Enumber = 'SE'
## motifExtension
motifExtension = 0
## working directory
projectFolder = f'./CRCmapper/2-1-TAD-TRAP.crc_macs2_HI-32_K27ac_narrow.05_broad.05_incltss_cutoff{Enumber}_ext{motifExtension}/'
projectName = 'macs2_HI-32_K27ac'
## SuperTable
superFile = './ROSE/2-1.rose_macs2_HI-32_K27ac_narrow.05_broad.05_incltss/H3K27ac_HI-32_peaks_AllEnhancers.table.txt'
superTable = parseTable(superFile, '\t')
## subpeak
subpeaks = './ChIP-seq/Meissner/macs2_HI-32_K27ac_narrow.05_broad.05/H3K27ac_HI-32_peaks.broadPeak'
## expressionTable
expressedNMfile = f'./CRCmapper/2-1.crc_macs2_HI-32_K27ac_narrow.05_broad.05_incltss_cutoff{Enumber}_ext500/H3K27ac_HI-32_EXPRESSED_TRANSCRIPTS.txt'
'''

'''
# 6-1-TAD-TRAP.crc_DE_narrow.05_broad.05_exclNMtss_cutoffSE_ext500
## enhancerNumber
Enumber = 'SE'
## motifExtension
motifExtension = 500
## working directory
projectFolder = f'./CRCmapper/6-1-TAD-TRAP.crc_DE_narrow.05_broad.05_exclNMtss_cutoff{Enumber}_ext{motifExtension}/'
projectName = 'DE'
## SuperTable
superFile = './ROSE/6.rose_DE_narrow.05_broad.05_exclNMtss/DE_peaks_AllEnhancers.table.txt'
superTable = parseTable(superFile, '\t')
## subpeak
subpeaks = './ChIP-seq/Meissner/macs2_DE_narrow.05_broad.05/DE_peaks.broadPeak'
## expressionTable
expressedNMfile = f'./CRCmapper/6-1.crc_DE_narrow.05_broad.05_exclNMtss_cutoff{Enumber}_ext{motifExtension}/DE_H3K27ac_EXPRESSED_TRANSCRIPTS.txt'
'''

'''
# 6-1-TAD-TRAP.crc_DE_narrow.05_broad.05_exclNMtss_cutoffSE_ext0
## enhancerNumber
Enumber = 'SE'
## motifExtension
motifExtension = 0
## working directory
projectFolder = f'./CRCmapper/6-1-TAD-TRAP.crc_DE_narrow.05_broad.05_exclNMtss_cutoff{Enumber}_ext{motifExtension}/'
projectName = 'DE'
## SuperTable
superFile = './ROSE/6.rose_DE_narrow.05_broad.05_exclNMtss/DE_peaks_AllEnhancers.table.txt'
superTable = parseTable(superFile, '\t')
## subpeak
subpeaks = './ChIP-seq/Meissner/macs2_DE_narrow.05_broad.05/DE_peaks.broadPeak'
## expressionTable
expressedNMfile = f'./CRCmapper/6-1.crc_DE_narrow.05_broad.05_exclNMtss_cutoff{Enumber}_ext500/DE_H3K27ac_EXPRESSED_TRANSCRIPTS.txt'
'''


'''# 1-1-TAD-TRAP. CRUP_HI-32_K27ac_incltss_cutoffSE_ext500
## enhancerNumber
Enumber = 'SE'
## motifExtension
motifExtension = 500
## working directory
projectFolder = f'./CRCmapper/1-1-TAD-TRAP.crc_CRUP_HI-32_K27ac_incltss_cutoff{Enumber}_ext{motifExtension}/'
projectName = 'CRUP_HI-32_K27ac'
## SuperTable
superFile = './ROSE/1-1.rose_CRUP_HI-32_K27ac_incltss/islets_AllEnhancers.table.txt'
superTable = parseTable(superFile, '\t')
## subpeak
subpeaks = './crup/islets.HI-32.CRUP.singleEnh.bed'
## expressionTable
expressedNMfile = f'./CRCmapper/1-1.crc_CRUP_HI-32_K27ac_incltss_cutoff{Enumber}_ext{motifExtension}/H3K27ac_HI-32_EXPRESSED_TRANSCRIPTS.txt'
'''

'''
# 1-1. CRUP_HI-32_K27ac_incltss_cutoffSE_ext0
## enhancerNumber
Enumber = 'SE'
## motifExtension
motifExtension = 0
## working directory
projectFolder = f'./CRCmapper/1-1-TAD-TRAP.crc_CRUP_HI-32_K27ac_incltss_cutoff{Enumber}_ext{motifExtension}/'
projectName = 'CRUP_HI-32_K27ac'
## SuperTable
superFile = './ROSE/1-1.rose_CRUP_HI-32_K27ac_incltss/islets_AllEnhancers.table.txt'
superTable = parseTable(superFile, '\t')
## subpeak
subpeaks = './crup/islets.HI-32.CRUP.singleEnh.bed'
## expressionTable
expressedNMfile = f'./CRCmapper/1-1.crc_CRUP_HI-32_K27ac_incltss_cutoff{Enumber}_ext{motifExtension}/H3K27ac_HI-32_EXPRESSED_TRANSCRIPTS.txt'
'''


"\n# 1-1. CRUP_HI-32_K27ac_incltss_cutoffSE_ext0\n## enhancerNumber\nEnumber = 'SE'\n## motifExtension\nmotifExtension = 0\n## working directory\nprojectFolder = f'./CRCmapper/1-1-TAD-TRAP.crc_CRUP_HI-32_K27ac_incltss_cutoff{Enumber}_ext{motifExtension}/'\nprojectName = 'CRUP_HI-32_K27ac'\n## SuperTable\nsuperFile = './ROSE/1-1.rose_CRUP_HI-32_K27ac_incltss/islets_AllEnhancers.table.txt'\nsuperTable = parseTable(superFile, '\t')\n## subpeak\nsubpeaks = './crup/islets.HI-32.CRUP.singleEnh.bed'\n## expressionTable\nexpressedNMfile = f'./CRCmapper/1-1.crc_CRUP_HI-32_K27ac_incltss_cutoff{Enumber}_ext{motifExtension}/H3K27ac_HI-32_EXPRESSED_TRANSCRIPTS.txt'\n"

## .1 createTADLoci

In [18]:
def createTADLoci(tadTable, tadType='domain'):
    '''
    takes as input a ROSE SuperEnhancer table 
    output a table of loci for SuperEnhancers
    '''

    print('CREATING TAD LOCUS COLLECTION')

    output = []
    if tadType == 'domain':
        
        count1 = 0
        count2 = 0
        count3 = 0
        count4 = 0
        
        
        for line in tadTable:
            if line[3] == 'domain':
                # extend the region
                if int(line[1]) >= 50000:
                    locus = Locus(line[0], int(line[1]) - 50000, int(line[2]) + 50000, '.', line[4], float(0))
                else:
                    locus = Locus(line[0], int(line[1]), int(line[2]) + 50000, '.', line[4], float(0))
                
                gapOverlaps = gapCollection.getOverlap(locus)
                
                if len(gapOverlaps) != 0:  
                    # if there's overlap between gap and extended domain, 
                    # then take the original domain without extension
                    # print(len(gapOverlaps))
                    # first recover the non-extended domain,
                    # then extend the domain until it reaches a gap.
                    minStart = int(locus.start()) + 50000
                    maxEnd = int(locus.end()) - 50000
                    
                    # 最小的tad start是原始domain start左侧 and extended domain start右侧这段区间中，最大的gap.end()
                    # 最大的tad end 是原始domain end右侧 and extended domain end左侧这段区间中，最小的gap.start()
                    minStart = max([int(gap.end()) for gap in gapOverlaps if int(gap.end()) <= minStart], default = minStart - 50000)
                    maxEnd = min([int(gap.start()) for gap in gapOverlaps if int(gap.start()) >= maxEnd], default = maxEnd + 50000)
                    
                    locus2 = Locus(line[0], minStart, maxEnd, '.', line[4], float(0))
                    # if locus != locus2:
                        # print(locus2.__str__())
                    locus = Locus(line[0], minStart, maxEnd, '.', line[4], float(0))
                    
                    boundaryOverlaps = boundaryCollection.getOverlap(locus)
                    
                    if len(boundaryOverlaps) != 0: 
                        # overlap with gaps and boudnaries
                        # if there's overlap between boundary and extended domain
                        # then extend the domain to include the overlapped boundary
                        
                        minStart = int(locus.start())
                        maxEnd = int(locus.end())
                        
                        # 最小tad start是与其重叠的boundary中，最小的boundary.start()
                        # 最大的tad end是与其重叠的boundary中，最大的boudnary.end()
                        # iteratively update start position of the region,
                        # to get the largest merged tad region
                        minStart = min([int(boundary.start()) for boundary in boundaryOverlaps if int(boundary.start()) < minStart], default = minStart)
                        maxEnd = max([int(boundary.end()) for boundary in boundaryOverlaps if int(boundary.end()) > maxEnd], default = maxEnd)
                        
                        locus = Locus(line[0], minStart, maxEnd, '.', line[4], float(0))
                        output.append(locus)
                        count1 += 1
                        
                        # if len(gapCollection.getOverlap(locus)) != 0:
                            # print('gap True, boundary True: ', locus.__str__(), len(gapCollection.getOverlap(locus)))
                    else:
                        # overlap with gaps, but not with boundaries
                        output.append(locus)
                        count2 += 1
                        # if len(gapCollection.getOverlap(locus)) != 0:
                            # print('gap True, boundary False: ', locus.__str__(), len(gapCollection.getOverlap(locus)))
                    
                else:
                    # if there's no overlap between gap and extended domain, 
                    # then take the extended domain
                    # print(locus)
                    boundaryOverlaps = boundaryCollection.getOverlap(locus)
                    
                    if len(boundaryOverlaps) != 0:
                        # not overlap with gaps, but with boundaries
                        # there's overlap between boundary and extended domain,
                        # then take the merged region
                        # print(len(boundaryOverlaps))
                        
                        minStart = int(locus.start())
                        maxEnd = int(locus.end())
                        
                        minStart = min([int(boundary.start()) for boundary in boundaryOverlaps if int(boundary.start()) < minStart], default = minStart)
                        maxEnd = max([int(boundary.end()) for boundary in boundaryOverlaps if int(boundary.end()) > maxEnd], default = maxEnd)
                        
                        locus = Locus(line[0], minStart, maxEnd, '.', line[4], float(0))
                        output.append(locus)
                        count3 += 1
                        # if len(gapCollection.getOverlap(locus)) != 0:
                            # print('gap False, boundary True: ', locus.__str__(), len(gapCollection.getOverlap(locus)))
                    
                    else:
                        # not overlap with gaps and boundaries, which means it overlaps with a domain
                        output.append(locus)
                        count4 += 1

        print('gap \t boudary \t #')
        print(f'True \t True \t {count1}')
        print(f'True \t False \t {count2}')
        print(f'False \t True \t {count3}')
        print(f'False \t False \t {count4}')
        print(f'total \t  \t {count1 + count2 + count3 + count4}')
        
        
    elif tadType == 'boundary':
        for line in tadTable:
            if line[3] == 'boundary':
                locus = Locus(line[0], line[1], line[2], '.', line[4], float(0))
                output.append(locus)
    elif tadType == 'gap':
        for line in tadTable:
            if line[3] == 'gap':
                locus = Locus(line[0], line[1], line[2], '.', line[4], float(0))
                output.append(locus)
    else:
        print('error!')
                
    return output

In [8]:
gapLoci = createTADLoci(tadTable, 'gap')
gapCollection = LocusCollection(gapLoci, 50)
print(*gapLoci[:5])
print(len(gapLoci))

CREATING TAD LOCUS COLLECTION
chr1(.):0-700000 chr1(.):3850000-3950000 chr1(.):13000000-13750000 chr1(.):24300000-24350000 chr1(.):29900000-30000000
206


In [9]:
boundaryLoci = createTADLoci(tadTable, 'boundary')
boundaryCollection = LocusCollection(boundaryLoci, 50)
print(*boundaryLoci[:5])
print(len(boundaryLoci))

CREATING TAD LOCUS COLLECTION
chr1(.):1300000-1650000 chr1(.):1650000-1850000 chr1(.):7800000-8000000 chr1(.):9650000-9950000 chr1(.):11850000-12150000
1081


In [10]:
tadLoci = createTADLoci(tadTable, 'domain')
tadCollection = LocusCollection(tadLoci, 50)
print(*tadLoci[:5])
print(len(tadLoci))

CREATING TAD LOCUS COLLECTION
gap 	 boudary 	 #
True 	 True 	 71
True 	 False 	 222
False 	 True 	 1213
False 	 False 	 1633
total 	  	 3139
chr1(.):700000-1650000 chr1(.):1650000-2650000 chr1(.):2550000-3400000 chr1(.):3300000-3850000 chr1(.):3950000-6050000
3139


In [ ]:
# stitch overlapped TAD
stitchedTADCollection = tadCollection.stitchCollection()
print(*list(stitchedTADCollection.getLoci())[:5])

chr1(.):700000-3850000 chr1(.):3950000-13000000 chr1(.):13750000-15650000 chr1(.):15850000-24300000 chr1(.):24350000-27600000


In [489]:
with open('./ChIP-seq/Meissner/hg19.chrom.sizes', 'r') as f:
    chrom = ['chr'+str(i) for i in range(1, 23)] + ['chrX']
    chromLen1 = 0
    lines = f.readlines()
    for i in range(0, len(lines)):
        line = lines[i].replace('\n', '').split('\t')
        if line[0] in chrom:
            chromLen1 += int(line[1])
            
    print(f'total hg19 chromSizes: \t {chromLen1:,}')

total hg19 chromSizes: 	 3,036,303,846


In [490]:
with open('./hg19.GSE63525_GM12878_50K.TopDom_10.bed', 'r') as f:
    chromLen2 = 0
    lines = f.readlines()
    for i in range(0, len(lines)):
        line = lines[i].replace('\n', '').split('\t')
        if line[3] != 'gap':
            chromLen2 += int(line[2]) - int(line[1])
            
    print(f'total hg19 TAD without gaps: \t {chromLen2: ,}')

total hg19 TAD without gaps: 	  2,803,136,782


In [491]:
# check the coverage of generated TAD
chromLen3 = 0
for loci in stitchedTADCollection.getLoci():
    chromLen3 += loci.coords()[1] - loci.coords()[0]
print(f'total TAD length: \t {chromLen3: ,}')

total TAD length: 	  2,770,736,782


In [496]:
print(f'TAD / all hg19 TAD: \t {chromLen3/chromLen2: .3f}')
print(f'TAD / hg19 genome: \t {chromLen3/chromLen1: .3f}')

TAD / all hg19 TAD: 	  0.988
TAD / hg19 genome: 	  0.913


In [33]:
def createSuperLoci(superTable, Enumber='super'):
    '''
    takes as input a ROSE SuperEnhancer table 
    output a table of loci for SuperEnhancers
    '''
    
    print('CREATING SUPER-ENHANCER LOCUS COLLECTION')
    
    output = []

    if Enumber == 'super':
        for line in superTable[6:]:
            if line[-1] == '1':
                locus = Locus(line[1], line[2], line[3], '.', line[0], (float(line[6])-float(line[7])))
                output.append(locus)
    else:
        end = 6+int(Enumber)
        for line in superTable[6:end]:
            locus = Locus(line[1], line[2], line[3], '.', line[0], (float(line[6])-float(line[7])))
            output.append(locus)

    return output

In [34]:
# superLoci = createSuperLoci(superTable)
superLoci = createSuperLoci(superTable)
print(*superLoci[:5])
len(superLoci)

CREATING SUPER-ENHANCER LOCUS COLLECTION
chr8(.):118136600-118286400 chr17(.):2689300-2887300 chr10(.):79044600-79291900 chr12(.):124837100-125053500 chr10(.):80818000-80957700


668

## .2 creatExpressionDict

In [61]:
# expressedNM = createExpressionDict(annotationFile, projectFolder, projectName, refseqToNameDict, expressionTable)
# expressedNMfile = './CRCmapper/1-1.crc_HI-32_K27ac_incltss_noExtension/H3K27ac_HI-32_EXPRESSED_TRANSCRIPTS.txt'
# expressedNMfile = './CRCmapper/1-1.crc_HI-32_K27ac_incltss/H3K27ac_HI-32_EXPRESSED_TRANSCRIPTS.txt'
expressedNMfile = './CRCmapper/2-2.crc_Beta_narrow.05_broad.1_incltss_cutoff500_ext500/Beta_EXPRESSED_TRANSCRIPTS.txt'
# expressedNMfile = f'./CRCmapper/3-1.crc_CRUP_HI-32_K27ac_incltss_peakCutoff.8_cutoffSE_ext500/H3K27ac_HI-32_EXPRESSED_TRANSCRIPTS.txt'
with open(expressedNMfile, "r") as f:
    expressedNM = [line.rstrip('\n') for line in f]
print(len(expressedNM))
print(expressedNMfile)

14739
./CRCmapper/2-2.crc_Beta_narrow.05_broad.1_incltss_cutoff500_ext500/Beta_EXPRESSED_TRANSCRIPTS.txt


In [62]:
expressedGenes = uniquify([refseqToNameDict[gene] for gene in expressedNM if gene in TFlist])
print(len(expressedGenes))

821


In [32]:
def createExpressionDict(annotationFile, projectFolder, projectName, refseqToNameDict,expressionTable):
    
    takes as input an activity table with refseq NMID in first column and expression or promoter
    acetylation level in a second column
    output a dictionary keyed by refseq containing activity

    print('CREATING EXPRESSION DICTIONARY')

    annotTable = parseTable(annotationFile, '\t')
    for line in annotTable:
        gid = line[1]
        genename = line[12].upper()
        refseqToNameDict[gid] = genename

    expresionFilename = projectFolder + 'matrix.gff'
    expressionTable = parseTable(expresionFilename, '\t')

    expressionDictNM = {}
    expressionDictGene = {}

    for line in expressionTable[1:]:
        trid = line[0]
        geneName = refseqToNameDict[trid]
        if len(expressionTable[1]) == 3: #when expressionTable is an output from bamToGFF.py
                exp = float(line[2])
        else: #when expressionTable is passed as an option (2 columns)
                exp = float(line[1])

        # Store the expression value for each NMid in a dict, keep higher value if multiple identical NMIDs
        if trid in expressionDictNM and exp > expressionDictNM[trid]:
            expressionDictNM[trid] = exp
        elif trid not in expressionDictNM:
            expressionDictNM[trid] = exp

        # Store the highest value of transcript expression for each gene
        if geneName in expressionDictGene and exp > expressionDictGene[geneName]:
            expressionDictGene[geneName] = exp
        elif geneName not in expressionDictGene:
            expressionDictGene[geneName] = exp
    
    expressedGenes = []
    expressedNM = []
    for trid in expressionDictNM:
        expressedGenes.append(refseqToNameDict[trid])
        expressedNM.append(trid)
    expressedGenes = uniquify(expressedGenes)
    
    return expressedNM, expressedGenes


In [33]:
expressedNM, expressedGenes = createExpressionDict(annotationFile, './CRCmapper/2-2.crc_Beta_narrow.05_broad.1_incltss_cutoff500/', projectName, refseqToNameDict, expressionTable)

CREATING EXPRESSION DICTIONARY


## .3 findCanidateTFs

In [22]:
def findCanidateTFs(annotationFile, tadLoci, expressedNM, TFlist, refseqToNameDict):
    '''
    find all TFs that are considered expressed within a TAD
    return a dictionary keyed by TAD loci that points to a list of TFs that are contained within the TAD
    '''

    print('FINDING CANIDATE TFs')
    
    startDict = makeStartDict(annotationFile)
    
    # Find the location of the TSS of all transcripts (NMid) considered expressed
    tssLoci = []
    for geneID in expressedNM:
        tssLoci.append(makeTSSLocus(geneID,startDict,0,0))  # 1bp region
    tssCollection = LocusCollection(tssLoci,50)

    # Assign all transcripts (NMid) that are TFs to a TAD if it is within the TAD
    tadAssignment = []
    tadAssignmentGene = []
    TFandTADdict = {}
    
    for tad in tadLoci:
        
        # Find all transcripts whose TSS occur within a TAD
        searchLocus = Locus(tad.chr(), tad.start(), tad.end(), '.')
        allEnhancerLoci = tssCollection.getOverlap(searchLocus)
        allEnhancerGenes = [locus.ID() for locus in allEnhancerLoci]
        
        geneName = uniquify([refseqToNameDict[gene] for gene in allEnhancerGenes if gene in TFlist])
        
        # Select the transcript if it is a TF, and allow for a TAD to have multiple TF genes inside
        if geneName != []:
            for gene in allEnhancerGenes:
                if gene in TFlist and tad not in TFandTADdict.keys():
                    TFandTADdict[tad] = [gene]
                elif gene in TFlist and tad in TFandTADdict.keys():
                    TFandTADdict[tad].append(gene)
            tadAssignment.append([tad.chr(), tad.start(), tad.end(), TFandTADdict[tad]])
            tadAssignmentGene.append([tad.chr(), tad.start(), tad.end(), geneName])
        else:
            TFandTADdict[tad] = []
    
    print(f'Number of TADs that contains TF genes: {len(tadAssignment)}, {len(tadAssignmentGene)}')
    

    return tadAssignment, tadAssignmentGene, TFandTADdict
        

In [23]:
# TAD domain
tadAssignment, tadAssignmentGene, TFandTADdict = findCanidateTFs(annotationFile, tadLoci, expressedNM, TFlist, refseqToNameDict)

FINDING CANIDATE TFs
Number of TADs that contains TF genes: 564, 564


In [25]:
print(len(tadAssignment))
tadAssignment[:5]

564


[['chr1', 700000, 1650000, ['NM_021170']],
 ['chr1', 2550000, 3400000, ['NM_199454']],
 ['chr1', 5950000, 6800000, ['NM_005341']],
 ['chr1', 6700000, 8000000, ['NM_001195563']],
 ['chr1', 7800000, 8500000, ['NM_001042682']]]

In [26]:
print(len(tadAssignmentGene))
tadAssignmentGene[:5]

564


[['chr1', 700000, 1650000, ['HES4']],
 ['chr1', 2550000, 3400000, ['PRDM16']],
 ['chr1', 5950000, 6800000, ['ZBTB48']],
 ['chr1', 6700000, 8000000, ['CAMTA1']],
 ['chr1', 7800000, 8500000, ['RERE']]]

In [27]:
print(len(TFandTADdict))
dict(list(TFandTADdict.items())[:5])

3139


{<__main__.Locus at 0x7fbff2e09750>: ['NM_021170'],
 <__main__.Locus at 0x7fc01216d4e0>: [],
 <__main__.Locus at 0x7fbff2e18be0>: ['NM_199454'],
 <__main__.Locus at 0x7fbff252be50>: [],
 <__main__.Locus at 0x7fbff252b790>: []}

In [28]:
# for visualization
# {'# of genes in a TAD': '# of TADs that contain this number of genes'}
countDict = {}
for line in tadAssignmentGene:
    l = len(uniquify(line[3]))
    if l > 5:
        print(line)
    if l not in countDict.keys():
        countDict[len(uniquify(line[3]))] = 1
    else:
        countDict[len(uniquify(line[3]))] += 1
        
countDict[0] = len(TFandTADdict) - len(tadAssignmentGene)

for k, v in sorted(countDict.items()):
    print(f'{k}: {v}, {v/len(tadAssignment):.2f}')

print(sum([countDict[i] for i in range(1,6)]))

['chr16', 3050000, 3600000, ['ZNF205', 'ZNF213', 'ZNF263', 'ZNF75A', 'ZNF174', 'ZNF200', 'ZNF597']]
['chr19', 8350000, 10200000, ['ZNF317', 'ZNF559', 'ZNF266', 'ZNF426', 'ZNF121', 'ZNF561', 'ZNF562']]
['chr19', 11350000, 13050000, ['ZNF627', 'ZNF441', 'ZNF491', 'ZNF440', 'ZNF69', 'ZNF700', 'ZNF136', 'JUNB', 'ZNF653', 'ZNF823', 'ZNF433', 'ZNF20', 'ZNF625', 'ZNF44', 'ZNF563', 'ZNF442', 'ZNF443', 'ZNF709', 'ZNF490']]
['chr19', 19650000, 24650000, ['ZNF101', 'ZNF93', 'ZNF90', 'ZNF85', 'ZNF430', 'ZNF714', 'ZNF431', 'ZNF493', 'ZNF257', 'ZNF726', 'ZNF254', 'PBX4', 'ZNF14', 'ZNF506', 'ZNF682', 'ZNF626', 'ZNF708', 'ZNF100', 'ZNF43', 'ZNF208', 'ZNF91', 'ZNF675', 'ZNF681']]
['chr19', 36650000, 38400000, ['ZNF146', 'ZNF382', 'ZNF567', 'ZNF568', 'ZNF420', 'HKR1', 'ZNF527', 'ZNF570', 'ZNF540', 'ZNF565', 'ZFP14', 'ZNF566', 'ZNF529', 'ZNF461', 'ZNF585A', 'ZNF569', 'ZNF571', 'ZFP30', 'ZNF607', 'ZNF573']]
['chr19', 43900000, 45350000, ['ZNF575', 'ZNF576', 'ZNF221', 'ZNF155', 'ZNF230', 'ZNF222', 'ZNF223'

In [29]:
# create new dict: tads with <= 5 expressed TF genes
# {tad: [geneNames]}
newTFandTADdict = {}

for tad, genes in TFandTADdict.items():
    if len(genes) != 0:
        geneNames = uniquify([refseqToNameDict[gene] for gene in genes if gene in TFlist])
        if not len(geneNames) > 5:
            newTFandTADdict[tad] = geneNames

print(f'# of TADs that contain <= 5 expressed TF genes: {len(newTFandTADdict.keys())}')

# of TADs that contain <= 5 expressed TF genes: 549


In [30]:
# create candidate gene list that contains gene NAMEs. 
candidateGenes = set()

for tad, genes in newTFandTADdict.items():
    
    # geneNames = uniquify([refseqToNameDict[gene] for gene in genes if gene in TFlist])
    intersect = candidateGenes.intersection(genes)
    
    # print genes that are located in overlapped TAD
    # if len(intersect) != 0:
        # print(f'{tad.__str__()} \t {len(intersect)} \t {intersect}')
    
    candidateGenes.update(genes)

print(f'# of expressed genes: \t {len(expressedGenes)}')
print(f'# of candidate genes: \t {len(candidateGenes)}')  # candidates是包含在TAD里的genes, 该TAD包含的genes不超过5个
print(f'# of non-recovered genes: \t {len(set(expressedGenes) ^ set(candidateGenes))}')
print(sorted(list(set(expressedGenes) ^ set(candidateGenes))))

# of expressed genes: 	 804
# of candidate genes: 	 590
# of non-recovered genes: 	 214
['ATF6B', 'CREB3L1', 'ELF3', 'ETV7', 'FEV', 'FIZ1', 'FOSL1', 'HKR1', 'JUNB', 'LBX2', 'MEF2D', 'MXD4', 'MZF1', 'NFKB2', 'PBX4', 'PEG3', 'RELA', 'RXRB', 'SIX5', 'SRY', 'TSC22D4', 'USF1', 'ZBTB12', 'ZBTB33', 'ZBTB45', 'ZFP14', 'ZFP28', 'ZFP30', 'ZFY', 'ZIK1', 'ZKSCAN1', 'ZKSCAN3', 'ZKSCAN4', 'ZKSCAN5', 'ZNF100', 'ZNF101', 'ZNF121', 'ZNF132', 'ZNF134', 'ZNF135', 'ZNF136', 'ZNF14', 'ZNF155', 'ZNF16', 'ZNF160', 'ZNF165', 'ZNF17', 'ZNF174', 'ZNF180', 'ZNF197', 'ZNF20', 'ZNF200', 'ZNF208', 'ZNF211', 'ZNF212', 'ZNF221', 'ZNF222', 'ZNF223', 'ZNF226', 'ZNF227', 'ZNF229', 'ZNF230', 'ZNF233', 'ZNF235', 'ZNF250', 'ZNF251', 'ZNF254', 'ZNF256', 'ZNF257', 'ZNF26', 'ZNF263', 'ZNF264', 'ZNF266', 'ZNF274', 'ZNF28', 'ZNF282', 'ZNF284', 'ZNF3', 'ZNF304', 'ZNF317', 'ZNF324', 'ZNF329', 'ZNF331', 'ZNF34', 'ZNF347', 'ZNF35', 'ZNF382', 'ZNF384', 'ZNF398', 'ZNF415', 'ZNF416', 'ZNF417', 'ZNF418', 'ZNF419', 'ZNF420', 'ZNF425', '

In [90]:
boundaryGeneT = set()
boundaryGeneF = set()

for line in boundaryAssignmentGene:
    intersect = candidateGenes.intersection(line[3])
    if len(intersect) != 0:
        boundaryGeneT.update(intersect)
    else:
        boundaryGeneF.update(line[3])

print(len(boundaryGeneT), boundaryGeneT)
print(len(boundaryGeneF), boundaryGeneF)

30 {'FOXA3', 'MLX', 'ZNF467', 'ZNF77', 'SRF', 'ZNF32', 'NFYA', 'ZBTB2', 'DBP', 'ESR2', 'IRF1', 'KLF16', 'RCOR3', 'ZNF484', 'ETV3', 'ZNF664', 'SNAI1', 'MYBL1', 'ATF6', 'GLIS3', 'ESRRA', 'ZFHX2', 'TEF', 'ZNF57', 'ZNF24', 'ZNF662', 'E2F5', 'KLF11', 'ATF3', 'ZNF485'}
94 {'IRF9', 'ZNF70', 'EGR4', 'IKZF4', 'NFIX', 'RARA', 'NOTO', 'NFKB2', 'IRF7', 'MESP2', 'MXD3', 'CREB3', 'FOXJ2', 'ATF5', 'ZNF653', 'TSC22D4', 'BATF2', 'ZNF410', 'FOXK1', 'NFE2L2', 'ZNF205', 'PPARD', 'ETV7', 'SIX5', 'NFYC', 'ZNF687', 'BCL6B', 'OVOL1', 'ZNF473', 'NFATC4', 'HHEX', 'NFX1', 'FEV', 'FOXK2', 'THRA', 'ZNF22', 'CREB3L1', 'WHSC1', 'JUND', 'MXD4', 'NR4A1', 'ZNF652', 'PREB', 'ZNF213', 'NKX3-1', 'GABPA', 'JUNB', 'ELF1', 'ZNF668', 'ID3', 'PRDM15', 'ZNF575', 'MEF2D', 'GRHL2', 'ADNP', 'FOXO4', 'RFX5', 'SCRT2', 'DDIT3', 'IRF3', 'ELK4', 'SNAPC4', 'NR1D1', 'ZNF384', 'PBX2', 'ZFP37', 'EGR1', 'RELA', 'DEAF1', 'ELF3', 'ID1', 'ATF7', 'SREBF2', 'ZNF576', 'HINFP', 'ZNF629', 'GMEB1', 'GLI1', 'LBX2', 'ZNF513', 'RXRB', 'ZNF646', 'ZSCAN2

---
## .4 generateTRAPsubPeaks

In [36]:
# subpeaks = SE subpeaks + remaining peaks in the TAD
def generateSubpeakFASTA(TFandTADdict, superLoci, subpeaks, genomeDirectory, projectName, projectFolder, motifExtension):
    '''
    keep TADs that overlap SE, and output peaks in the TADs
    '''
    print('MAKE FASTA')
    
    # make bed file
    subpeakDict = {}
    subpeakBED = []
    subpeakTable = parseTable(subpeaks, '\t')
    
    # subpeakCollection contains all peaks called from macs2
    subpeakLoci = [Locus(l[0], int(l[1]), int(l[2]), '.') for l in subpeakTable]
    subpeakCollection = LocusCollection(subpeakLoci, 50)  # macs2 broadPeaks
    
    # tadLoci contains TADs that have 1-5 TF genes
    tadLoci = list(TFandTADdict.keys())
    tadCollection = LocusCollection(tadLoci, 50)
    print(len(tadCollection.getLoci()))
    
    # subTADloci contains all TAD regions that have at least 1 TF gene
    # subTADloci = [tad for tad in TFandTADdict.keys() if len(TFandTADdict[tad]) != 0]  # len(subTADloci)
    # subTADcollection = LocusCollection(subTADloci, 50)
    
    used = {}
    
    for loci in superLoci:
        overlappedTAD = list(tadCollection.getOverlap(loci))  # subTAD regions that fully contain 1 or more superLoci
        
        if not overlappedTAD: continue;  # if overlappedSE is empty 如果SE没有和任何TAD重叠
            
        else:  # if such tad exists
            for tad in overlappedTAD:  # for loop is for the case that one SE overlap two or more TADs
                if tad in used.keys(): 
                    print(loci.__str__(), tad.__str__())
                    # continue;  # uniq subTAD_containing_SE
                
                # used[subTAD] = None
                for gene in TFandTADdict[tad]:
                    if gene not in subpeakDict.keys():
                        subpeakDict[gene] = set()
                    overlaps = set(subpeakCollection.getOverlap(tad))  # broadPeaks that overlap the TAD
                    overlaps.update(subpeakCollection.getOverlap(loci))  # plus peaks that overlap the SE
                    extendedOverlaps = [makeSearchLocus(x, motifExtension, motifExtension) for x in overlaps]  
                    # extends the overlapped broadPeaks by motifExtension
                    
                    overlapCollectionTemp = LocusCollection(extendedOverlaps, 50)
                    overlapCollection = overlapCollectionTemp.stitchCollection()  # links the overlapped broadPeaks that overlaps each other
                    
                    for overlap in overlapCollection.getLoci():
                        pos = [overlap.chr(), overlap.start(), overlap.end()]
                        if pos not in subpeakBED:
                            subpeakBED.append(pos)
                        subpeakDict[gene].add(overlap)
                    
    print(f'# of subpeaks: {len(subpeakBED)}')
    '''
    subpeakBED_uniq_set = set()
    subpeakBED_uniq = []
    for line in subpeakBED:
        pos = f'{line[0]}:{line[1]}-{line[2]}'
        if pos not in subpeakBED_uniq_set:
            subpeakBED_uniq.append(line)
            subpeakBED_uniq_set.add(pos)
    
    print(f'# of uniq subpeaks: {len(subpeakBED_uniq)}')
    '''
    
    if motifExtension != 0:
        bedfilename = projectFolder + projectName + '_subpeaks_ext500.bed'
    else:
        bedfilename = projectFolder + projectName + '_subpeaks_ext0.bed'

                    
    
    # make fasta file
    # fasta = []
    
    # for gene in subpeakDict:
        # for subpeak in subpeakDict[gene]:
            
            # fastaTitle = gene + '|'  + subpeak.chr() + '|' + str(subpeak.start()) + '|' + str(subpeak.end())
            # fastaLine = fetchSeq(genomeDirectory, subpeak.chr(), int(subpeak.start()+1), int(subpeak.end()+1))
            
            # fasta.append('>' + fastaTitle)
            # fasta.append(fastaLine.upper())
    
    # outname = projectFolder + projectName + '_SUBPEAKS.fa'
    # unParseTable(fasta, outname, '')
    # print(f'writing subpeakDict to {outname}')
     
    unParseTable(subpeakBED, bedfilename, '\t')
    
    return subpeakDict, subpeakBED

In [1105]:
# subpeakDict: TF_NMid: SE constituents
print(f'motifExtension: {motifExtension}')
subpeakDict, subpeakBED = generateSubpeakFASTA(newTFandTADdict, superLoci, subpeaks, genomeDirectory, projectName, projectFolder, motifExtension)

motifExtension: 500
MAKE FASTA
549
# of subpeaks: 7925
write table to ./CRCmapper/1-1-TAD-TRAP.crc_CRUP_HI-32_K27ac_incltss_cutoffSE_ext500/CRUP_HI-32_K27ac_subpeaks_ext500.bed


In [37]:
# subpeakDict: TF_NMid: SE constituents
print(f'motifExtension: {motifExtension}')
subpeakDict0, subpeakBED0 = generateSubpeakFASTA(newTFandTADdict, superLoci, subpeaks, genomeDirectory, projectName, projectFolder, 0)

motifExtension: 0
MAKE FASTA
549
# of subpeaks: 5223
write table to ./CRCmapper/3-1-TAD-TRAP.crc_CRUP_HI-32_K27ac_incltss_peakCutoff.8_cutoffSE_ext0/CRUP_HI-32_K27ac_subpeaks_ext0.bed


## .5 TRAP

In [11]:
import re

In [12]:
# create TRAP matrix: candidateMotifs.fa
with open('/project/ngsvin/bin/TRAP/Data/merged_matrices.TFP_2022.2.fa', 'r') as f:
    lines = f.readlines()
    # mat = open('/project/NeuralNet/CRC/CRCmapper/2-2-TRAP.crc_Beta_narrow.05_broad.1_incltss_cutoff500/Beta_canidateMotifs.fa', 'w')
    mat = open(projectFolder+projectName+'_canidateMotifs.fa', 'w')
    for i in range(0, len(lines)): # len(lines)
        if lines[i].startswith('>V'):
            r = re.compile('/gene', flags=re.IGNORECASE)
            if re.findall(r, lines[i]):
                r = re.compile('|'.join(candidateGenes2), flags=re.IGNORECASE)
                if re.findall(r, lines[i]):
                    # print(lines[i], end="")
                    mat.writelines(lines[i])
                    j = i+1
                    while not lines[j].startswith('>'):
                        # print(lines[j], end="")
                        mat.writelines(lines[j])
                        j += 1
                        if j >= len(lines):
                            break;
    mat.close()


NameError: name 'candidateGenes2' is not defined

In [13]:
with open('/project/ngsvin/bin/TRAP/Data/merged_matrices.TFP_2022.2.fa', 'r') as f:
    motif2TFdict = {}
    lines = f.readlines()
    for i in range(0, len(lines)): # len(lines)
        if lines[i].startswith('>V'):
            r = re.compile('/gene', flags=re.IGNORECASE)
            if re.findall(r, lines[i]):
                line = lines[i].replace('\n', '').split(' /')
                r = re.compile('name')
                name = list(filter(r.match, line))[0].split('=')[1]
                r = re.compile('gene')
                gene = list(filter(r.match, line))[0].split('=')[1]
                if gene not in motif2TFdict.keys():
                    motif2TFdict[gene] = []
                    motif2TFdict[gene].append(name)
                else:
                    motif2TFdict[gene].append(name)
# 1490 TF with 7353 motifs


In [1112]:
print(len(subpeakDict.keys()))  # number of candidate TF 2

308


In [1113]:
# for checking the number of motifs
count = 0
TFcount = 0
for i in candidateGenes2:
    if i in motif2TFdict.keys():
        count += len(motif2TFdict[i])
        TFcount += 1
    else:
        print(i)
print(f'# of total motifs: {count}')
print(f'# of TF that has motifs: {TFcount}')

RFX8
PREB
EAF2
TOX3
MYT1
TUB
ARID1B
BATF2
UBTF
SIM1
WHSC1
MYT1L
HOPX
ZBTB25
MXD1
CAMTA2
GRHL3
CREBL2
ID3
ZBTB8A
CIC
ADNP
NPAS4
TSC22D1
HHEX
ZKSCAN2
PGR
ID1
RERE
ZNF589
TSHZ1
HES4
RCOR2
ARID1A
WIZ
IKZF4
# of total motifs: 1812
# of TF that has motifs: 272


In [ ]:
# run TRAP on shell
TRAP='/project/ngsvin/bin/TRAP/TRAPv1.04'
genome='/project/NeuralNet/CRC/ChIP-seq/mapping/bowtie2/hg19/hg19.fa'
norm='/project/ngsvin/bin/TRAP/Data/hg19.TSSup.TFP_2022.2.GEVparams'
folder='/project/NeuralNet/CRC/CRCmapper/2-2-TRAP.crc_Beta_narrow.05_broad.1_incltss_cutoff500_ext500/Beta_'
matrix=${folder}'canidateMotifs.fa'
region=${folder}'subpeaks_ext500.bed'
$TRAP -thread 20 -s $genome -norm $norm -matrix $matrix -region $region -gene > ${folder}TRAPresults_ext500.bed &
$TRAP -thread 10 -s $genome -norm $norm -matrix $matrix -region $region -gene -w 5000 > ${folder}TRAPresults_ext500_win5k.bed &

## .6 findAuto

In [42]:
import pandas as pd
import numpy as np
from math import ceil, log
import networkx as nx
from networkx.algorithms.clique import find_cliques_recursive

In [38]:
def posTF(subpeakDict):
    posTFdict = {}
    for TF in subpeakDict.keys():
        for peak in subpeakDict[TF]:
            peakPos = f'{peak.chr()}:{peak.start()}-{peak.end()}'
            if peakPos in posTFdict.keys():
                # print(peakPos)
                posTFdict[peakPos].append(TF)
            else:
                posTFdict[peakPos] = [TF]

    print(len(posTFdict.keys()))
    print(dict(list(posTFdict.items())[0:5]))
    
    return posTFdict

In [39]:
#posTFdict = posTF(subpeakDict)
posTFdict = posTF(subpeakDict0)


5223
{'chr17:2787000-2789200': ['MNT'], 'chr17:3721700-3722800': ['MNT'], 'chr17:2340800-2343000': ['MNT', 'HIC1'], 'chr17:2356200-2357300': ['MNT', 'HIC1'], 'chr17:1882600-1883700': ['MNT', 'HIC1']}


In [40]:
# {gene: other genes in the same TAD}
ggDict = {}
for genes in posTFdict.values():
    for gene in genes:
        if gene not in ggDict.keys():
            ggDict[gene] = ','.join(genes)
print(dict(list(ggDict.items())[0:5]))

{'MNT': 'MNT', 'HIC1': 'MNT,HIC1', 'ZNF664': 'ZNF664', 'SREBF1': 'SREBF1', 'CDC5L': 'CDC5L'}


In [1119]:
TRAPfile = './CRCmapper/1-1-TAD-TRAP.crc_CRUP_HI-32_K27ac_incltss_cutoffSE_ext0/' + projectName + '_TRAPresults_ext' + str(0) + '.bed'


In [44]:
TRAPfile = projectFolder + projectName + '_TRAPresults_ext' + str(motifExtension) + '.bed'
TRAPdf = pd.read_table(TRAPfile, skiprows=1, header=None, sep='\t', names=['peak_chr', 'peak_start', 'peak_end', 'motif', 'p_value'])
TRAPdf = TRAPdf.astype({'peak_start': int, 'peak_end': int, 'p_value': float})
TRAPdf.head(5)

,peak_chr,peak_start,peak_end,motif,p_value
0,chr1,204052701,204053800,V$AFP1_Q6#ZFHX3,0.536265
1,chr1,204052701,204053800,V$AP4_01#TFAP4,0.353191
2,chr1,204052701,204053800,V$AP4_Q5#TFAP4,0.191597
3,chr1,204052701,204053800,V$AP4_Q6#TFAP4,0.127290
4,chr1,204052701,204053800,V$AP4_Q6_01#TFAP4,0.114891


In [45]:
TRAPdf['peak'] = TRAPdf[TRAPdf.columns[:3]].apply(lambda x: f'{x[0]}:{x[1]-1}-{x[2]}', axis=1)
TRAPdf[['motif_name', 'motif_gene']] = TRAPdf.motif.str.split('#', expand=True)
TRAPdf['peak_gene'] = TRAPdf.peak.apply(lambda x: ','.join(posTFdict[x]))
TRAPdf = TRAPdf[['peak', 'peak_gene', 'motif_name', 'motif_gene', 'p_value']]
TRAPdf.head()

,peak,peak_gene,motif_name,motif_gene,p_value
0,chr1:204052700-204053800,SOX13,V$AFP1_Q6,ZFHX3,0.536265
1,chr1:204052700-204053800,SOX13,V$AP4_01,TFAP4,0.353191
2,chr1:204052700-204053800,SOX13,V$AP4_Q5,TFAP4,0.191597
3,chr1:204052700-204053800,SOX13,V$AP4_Q6,TFAP4,0.127290
4,chr1:204052700-204053800,SOX13,V$AP4_Q6_01,TFAP4,0.114891


In [50]:
gene = 'NKX6-1'
TRAPdf[TRAPdf.motif_gene == gene].sort_values(by = 'p_value', ignore_index = True).query("peak_gene == @ggDict[@gene]", engine='python').head(10)


,peak,peak_gene,motif_name,motif_gene,p_value
27,chr4:85354900-85356000,NKX6-1,V$NKX61_02,NKX6-1,0.000103
194,chr4:85412800-85413900,NKX6-1,V$NKX61_01,NKX6-1,0.003127
225,chr4:85356400-85357500,NKX6-1,V$NKX61_02,NKX6-1,0.003698
276,chr4:85354900-85356000,NKX6-1,V$NKX61_08,NKX6-1,0.004600
293,chr4:85356400-85357500,NKX6-1,V$NKX61_07,NKX6-1,0.004921
353,chr4:85356400-85357500,NKX6-1,V$NKX61_03,NKX6-1,0.005840
603,chr4:85364100-85365200,NKX6-1,V$NKX61_03,NKX6-1,0.010696
626,chr4:85354900-85356000,NKX6-1,V$NKX61_07,NKX6-1,0.011121
728,chr4:85411600-85412700,NKX6-1,V$NKX61_09,NKX6-1,0.013128
794,chr4:85377800-85378900,NKX6-1,V$NKX61_10,NKX6-1,0.014696


In [49]:
TRAPdf[TRAPdf.motif_gene == gene].sort_values(by = 'p_value', ignore_index = True).shape


(36561, 5)

In [48]:
gene = 'NKX6-1'
TRAPdf[TRAPdf.motif_gene == gene].sort_values(by = 'p_value', ignore_index = True).query("peak_gene == @ggDict[@gene]", engine='python').groupby('peak')['p_value'].min().reset_index().sort_values(by = 'p_value', ignore_index = True)


,peak,p_value
0,chr4:85354900-85356000,0.000103
1,chr4:85412800-85413900,0.003127
2,chr4:85356400-85357500,0.003698
3,chr4:85364100-85365200,0.010696
4,chr4:85411600-85412700,0.013128
5,chr4:85377800-85378900,0.014696
6,chr4:85342100-85343200,0.021309
7,chr4:85353700-85354800,0.037722
8,chr4:84469200-84470300,0.038683
9,chr4:85338100-85339200,0.039896


In [1149]:
#percent = 0.001
#percent = 0.0025
percent = 0.005
#percent = 0.01
#percent = 0.02
#percent = 0.04
#percent = 0.05
autoTF = set()
pairDict = {}
count = 0
for TF in candidateGenes2:
    TFdf = TRAPdf[TRAPdf.motif_gene == TF].sort_values(by='p_value', ignore_index=True)
    listLen = TFdf.shape[0]
    if listLen != 0:
        cutoff = ceil(listLen * percent)
        selfdf = TFdf.query("peak_gene == @ggDict[@TF]", engine='python')
        selfRank = selfdf.index[0]  + 1
        selfP = selfdf['p_value'].values[0]
        
        i = 0
        while selfP == 0.0:
            selfP = selfdf['p_value'].values[i+1]
            selfRank = selfdf.index[i+1] + 1
            i += 1
        print(TF, listLen, selfRank, selfP, sep=',')
        
        autoTF.add(TF)
        pairdf = TFdf[:selfRank]
        count += len(uniquify(pairdf.peak_gene))
            
        for i, TF2 in enumerate(pairdf.peak_gene):
                if pairdf.p_value[i] == 0.0:
                    continue;
                if (TF, TF2) not in pairDict.keys():
                    pairDict[(TF, TF2)] = pairdf.p_value[i]
        
        if (selfRank <= cutoff) and (selfP != 0.0):
            autoTF.add(TF)
            # print(TF, cutoff, selfRank, selfP, sep=',')
            
            pairdf = TFdf[:cutoff]
            # count += len(uniquify(pairdf.peak_gene))
            count += len(uniquify([TF for TFlist in pairdf.peak_gene.apply(lambda x: x.split(',')).values for TF in TFlist]))

            
            for i, TFls in enumerate(pairdf.peak_gene.apply(lambda x: x.split(',')).values):
                if pairdf.p_value[i] == 0.0:
                    # print((TF, TF2, pairdf.p_value[i]))
                    continue;
                for TF2 in TFls:
                    if (TF, TF2) not in pairDict.keys():
                        pairDict[(TF, TF2)] = pairdf.p_value[i]
                    
print(f'len(autoTF): {len(autoTF)}')
print(f'len(edges): {count}, {len(pairDict.keys())}')

FOSL2,106968,17,0.00132844
ELF1,280791,638,0.00716889
NKX2-2,80226,484,0.00457029
ZNF687,13371,639,0.0471801
XBP1,120339,1147,0.00929125
SCRT2,93597,848,0.0086011
ZBTB11,40113,237,0.00518569
ZNF785,13371,37,0.00134973
FOXK1,147081,795,0.0053527
ISL1,120339,48,0.00240391
ZNF513,40113,50,0.000632387
E2F2,120339,88,0.00107724
ZBTB17,40113,16,0.00140586
DBP,106968,1098,0.00736771
KLF9,66855,527,0.0209501
NR1D1,66855,52,0.000556938
ETS1,187194,1098,0.0123001
ZNF707,13371,67,0.0020391
FOXP4,13371,102,0.00865321
SP1,267420,940,0.0256598
ZBTB7B,80226,66,0.002233
ZNF182,40113,123,0.000987638
STAT6,93597,519,0.00555898
EGR4,93597,238,0.00503014
ZNF668,26742,1355,0.0488445
JUN,147081,737,0.00450663
YY1,254049,1287,0.00667287
TEF,160452,705,0.00322107
ZNF526,26742,128,0.00827388
NR2F6,133710,364,0.00165618
KLF13,106968,130,0.00773248
ZNF575,13371,7671,0.684752
BHLHE40,160452,4436,0.027605
ZFHX2,13371,188,0.0110161
ZNF646,40113,136,0.00379208
FOS,227307,2239,0.0071246
EGR3,147081,64,0.00168975
ARID

In [1126]:
edgeList = []
for pair in pairDict.keys():
    if (pair[0] in autoTF) and (pair[1] in autoTF):
        edgeList.append((pair[0], pair[1], -log(pairDict[pair])))
        #edgeList.append((pair[0], pair[1]))
print(len(edgeList))
print(len(set(edgeList)))
print(edgeList[:10])

38440
38440
[('FOSL2', 'HNF4A', 9.546687616409244), ('FOSL2', 'ARID5B', 7.350229347971895), ('FOSL2', 'SOX9', 7.33873667630915), ('FOSL2', 'HES1', 7.183118869768971), ('FOSL2', 'NR3C1', 7.153416715571305), ('FOSL2', 'CEBPB', 6.961418748437814), ('FOSL2', 'MEIS3', 6.759404241759336), ('FOSL2', 'FOXK1', 6.7288646009276425), ('FOSL2', 'MAFK', 6.679504790299783), ('FOSL2', 'HIF1A', 6.675334890887718)]


In [ ]:
graph = nx.DiGraph()
graph.add_nodes_from(autoTF)
graph.add_weighted_edges_from(edgeList)

autoRegNodeList = set()
autoRegEdgeList = set()
G = nx.DiGraph(name='Beta_narrow.05_broad.1_incltss_cutoff500_ext0_TRAP_noTAD')

for n in autoTF:
    for m in autoTF:
        if n == m:
            autoRegNodeList.add(n)
            autoRegEdgeList.add((n, m, graph.edges[n, m]['weight']))
        else:
            if graph.has_edge(n,m) and graph.has_edge(m,n):
                autoRegNodeList.add(n)
                autoRegEdgeList.add((n, m, graph.edges[n, m]['weight']))
                autoRegEdgeList.add((m, n, graph.edges[m, n]['weight']))
                
G.add_nodes_from(autoRegNodeList)
G.add_weighted_edges_from(autoRegEdgeList)

cliques = nx.find_cliques_recursive(G)
cliqueList = list(cliques)

cliqueRanking = []

for clique in cliqueList:
    cliqueScore = 0
    edgeNum = 0
    for n in clique:
        for m in clique:
            if n == m:
                cliqueScore += G.edges[n, m]['weight']
                edgeNum += 1
            else:
                cliqueScore += G.edges[n, m]['weight']
                cliqueScore += G.edges[m, n]['weight']
                edgeNum += 2
                
    cliqueRanking.append((clique, cliqueScore/edgeNum, len(clique)))
    
sortCliqueRanking = sorted(cliqueRanking, reverse=True, key=lambda x:x[1])
sortCliqueRanking[:5]

In [1137]:
sortCliqueRanking[:5]

[(['MEIS2', 'ZBTB34', 'MNX1', 'NKX6-1', 'HOMEZ'], 7.883992341635171, 5),
 (['MEIS2', 'ZBTB34', 'MNX1', 'FOXP2'], 7.741351049477134, 4),
 (['MEIS2', 'FOXO1', 'ZNF276', 'HOMEZ', 'MNX1', 'NR3C1'],
  7.438661155350288,
  6),
 (['MEIS2', 'FOXO1', 'ZNF276', 'FOXP2', 'MNX1', 'MEIS1', 'NR3C1'],
  7.338887034328841,
  7),
 (['MEIS2', 'FOXO1', 'ELF1', 'HOMEZ', 'FOXK1', 'MNX1', 'NKX6-1', 'ZNF77'],
  7.313715000754778,
  8)]

In [1132]:
edges = [[edge_data[0], edge_data[1], edge_data[-1]["weight"]] for edge_data in G.edges(data=True)]
len(edges)

32214

In [1146]:
if percent >= 0.01:
    autoTFFile = projectFolder + projectName + f'_AUTOREG_top{int(percent*100)}%.txt'
    cliqueFile = projectFolder + projectName + f'_CRC_SCORES_top{int(percent*100)}%.txt'
    edgeFile = projectFolder + projectName + f'_EDGES_top{int(percent*100)}%.txt'
else:
    autoTFFile = projectFolder + projectName + f'_AUTOREG_top{percent*100}%.txt'
    cliqueFile = projectFolder + projectName + f'_CRC_SCORES_top{percent*100}%.txt'
    edgeFile = projectFolder + projectName + f'_EDGES_top{percent*100}%.txt'

unParseTable(sorted(autoTF), autoTFFile, '')
unParseTable(sortCliqueRanking, cliqueFile, '\t')
unParseTable(edges, edgeFile, '\t')

write table to ./CRCmapper/1-1-TAD-TRAP.crc_CRUP_HI-32_K27ac_incltss_cutoffSE_ext500/CRUP_HI-32_K27ac_CRC_SCORES_top1%_cross.txt
